In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Float
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
class Surfer(Base):
    __tablename__ = "surfers"
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    hometown = Column(String(255))
    wipeouts = Column(Integer)
    rank = Column(Integer)


class Board(Base):
    __tablename__ = "surfboards"
    id = Column(Integer, primary_key=True)
    surfer_id = Column(Integer)
    board_name = Column(String(255))
    color = Column(String(255))
    length = Column(Integer)

In [4]:
# dir(Base)

In [5]:
# Base.__dict__

In [6]:
surfer = Surfer(name = "John", hometown = "houston", rank = 12)

In [7]:
surfer.rank

12

In [8]:
board = Board(surfer_id = 7, board_name = "glider", color = "green", length = 5)

In [9]:
board.board_name

'glider'

In [10]:
engine = create_engine("sqlite:///./surfer_board.sqlite")

In [11]:
conn = engine.connect()

In [12]:
Base.metadata.create_all(conn)

In [13]:
from sqlalchemy.orm import Session

In [14]:
session = Session(bind = engine)

In [15]:
session.add(surfer)
session.add(board)
session.commit()

In [16]:
surfer_list = session.query(Surfer)

In [19]:
for x in surfer_list:
    print(x.name)
    print(x.hometown)

John
houston


In [20]:
surfer2 = Surfer(name = "Maria", hometown = "Seatle", rank = 3)

In [21]:
session.add(surfer2)
session.commit()

In [24]:
for x in surfer_list:
    print(x.name)
    print(x.hometown)

John
houston
Maria
Seatle


In [27]:
for x in session.query(Surfer).filter(Surfer.hometown == "Seatle"):
    print(x.name)

Maria


In [28]:
session.query(Surfer).filter(Surfer.hometown == "Seatle").count()

1

In [29]:
surfer3 = Surfer(name = "Jannie", hometown = "Seatle", rank = 7)

In [30]:
session.add(surfer3)
session.commit()

In [31]:
session.query(Surfer).filter(Surfer.hometown == "Seatle").count()

2

In [32]:
for x in session.query(Surfer).filter(Surfer.hometown == "Seatle"):
    print(x.name)

Maria
Jannie


In [34]:
seatle_rankings = []

for x in session.query(Surfer).filter(Surfer.hometown == "Seatle"):
    seatle_rankings.append(x.rank)

In [35]:
seatle_rankings

[3, 7]

In [37]:
def returning_rankings_by_city(city_name = None):
    return [x.rank for x in session.query(Surfer).filter(Surfer.hometown == city_name)]

In [40]:
returning_rankings_by_city("Seatle")

[3, 7]

In [41]:
import pandas as pd
from sqlalchemy import inspect

In [42]:
inspector = inspect(engine)

In [43]:
inspector.get_table_names()

['surfboards', 'surfers']

In [44]:
pd.read_sql_table(table_name = "surfers", con = engine)

,id,name,hometown,wipeouts,rank
0,1,John,houston,None,12
1,2,Maria,Seatle,None,3
2,3,Jannie,Seatle,None,7


In [49]:
query = """
select *
from surfers
where hometown = 'Seatle'

"""

pd.read_sql(sql = query, con = engine)

,id,name,hometown,wipeouts,rank
0,2,Maria,Seatle,None,3
1,3,Jannie,Seatle,None,7


# ex 2

In [50]:
engine2 = create_engine("sqlite:///./sunshine.sqlite")

In [51]:
inspector = inspect(engine2)

In [52]:
inspector.get_table_names()

['sunshine']

In [54]:
pd.read_sql_table("sunshine", con = engine2).head(2)

,id,Country,City,Jan,Apr,Jul,Oct,Year
0,0,Afghanistan,Kabul,177.2,232.5,356.8,282.6,3175.1
1,1,Albania,Tirana,124.0,191.0,354.0,218.0,2544.0


In [55]:
class Sunshine(Base):
    __tablename__ = "sunshine"
    id = Column(Integer, primary_key = True)
    Country = Column(String)
    City = Column(String)
    Jan = Column(Float)
    Apr = Column(Float)
    Jul = Column(Float)
    Oct = Column(Float)
    Year = Column(Float)

In [56]:
session2 = Session(bind = engine2)

In [59]:
for x in session2.query(Sunshine).filter(Sunshine.Country == "Canada"):
    print(x.City)

Calgary
Churchill
Edmonton
Iqaluit
Montreal
Toronto
Vancouver
Whitehorse
Winnipeg


In [60]:
def returning_city_names(city_name = None):
    return [x.City for x in session2.query(Sunshine).filter(Sunshine.Country == city_name)]

In [62]:
len(returning_city_names("Canada"))

9

In [63]:
session2.query(Sunshine).filter(Sunshine.Country == "Canada").count()

9

In [70]:
for x in session2.query(Sunshine).filter(Sunshine.Country == "Canada").first():
    print(x.City) # !!! why 

TypeError: 'Sunshine' object is not iterable

In [72]:
session2.query(Sunshine).filter(Sunshine.Country == "Canada").first().City

'Calgary'

In [73]:
session2.query(Sunshine).filter(Sunshine.Country == "Canada").first().Apr

220.2

In [ ]:
# update

# instead of writing column names to the end of it one at a time, return it all , 
# in other words, return the first result, first row of the filtering

In [179]:
session2_canada = session2.query(Sunshine).filter(Sunshine.Country == "Canada").first()

In [180]:
session2_canada.Apr

198.2

In [182]:
session2_canada.Apr = session2_canada.Apr + 700

In [183]:
session2.dirty
session2.commit()

In [184]:
for x in session2.query(Sunshine).filter(Sunshine.Country == "Canada"):
    print(f"{x.City}, {x.Apr}")
# !!!

Churchill, 898.2
Edmonton, 244.2
Iqaluit, 216.5
Montreal, 178.0
Toronto, 180.0
Vancouver, 185.0
Whitehorse, 238.5
Winnipeg, 241.4


In [120]:
session2.query(Sunshine).filter(Sunshine.City == "Calgary").one().Oct

175.4

In [121]:
# delete

session2.delete(session2.query(Sunshine).filter(Sunshine.City == "Calgary").one())

In [122]:
session2.commit()

In [123]:
session2.query(Sunshine).filter(Sunshine.City == "Calgary").all()

[]

In [185]:
query = """
select *
from sunshine
where Country = 'Canada'

"""

pd.read_sql(sql = query, con = engine2)

,id,Country,City,Jan,Apr,Jul,Oct,Year
0,54,Canada,Churchill,79.7,898.2,281.7,58.1,1799.5
1,55,Canada,Edmonton,100.8,244.2,307.5,170.8,2344.8
2,56,Canada,Iqaluit,32.4,216.5,236.8,51.4,1476.8
3,57,Canada,Montreal,101.0,178.0,272.0,144.0,2051.0
4,58,Canada,Toronto,85.9,180.0,279.6,154.3,2066.4
5,59,Canada,Vancouver,60.1,185.0,289.8,120.7,1937.6
6,60,Canada,Whitehorse,43.8,238.5,247.6,84.9,1827.1
7,61,Canada,Winnipeg,114.7,241.4,308.3,147.4,2352.9


In [146]:
from sqlalchemy import desc

In [147]:
session2.query(Sunshine).order_by(desc(Sunshine.Year)) # descending !!!

In [148]:
# order by

for x in session2.query(Sunshine).order_by(desc(Sunshine.Year)).all():
    print(f"{x.id}, {x.Country}, {x.City}, {x.Year}")

367, United States, Yuma, 4015.3
109, Egypt, Marsa Alam, 3958.0
107, Egypt, Dakhla Oasis, 3943.4
67, Chile, Calama, 3926.2
349, United States, Phoenix, 3871.6
212, Namibia, Keetmanshoop, 3870.0
335, United States, Las Vegas, 3825.3
362, United States, Tucson, 3806.0
328, United States, El Paso, 3762.5
283, Sudan, Khartoum, 3737.1
273, South Africa, Upington, 3731.8
3, Algeria, Tamanrasset, 3686.0
190, Madagascar, Toliara, 3610.0
354, United States, Sacramento, 3607.8
213, Namibia, Windhoek, 3605.0
177, Kenya, Lodwar, 3582.0
105, Egypt, Alexandria, 3579.5
32, Botswana, Ghanzi, 3579.0
329, United States, Fresno, 3564.2
106, Egypt, Cairo, 3541.8
108, Egypt, Hurghada, 3530.7
311, United ArabEmirates, Dubai, 3508.7
64, Chad, Abéché, 3500.9
13, Australia, Alice Springs, 3499.1
231, Oman, Muscat, 3493.3
209, Morocco, Ouarzazate, 3416.4
314, United States, Albuquerque, 3415.4
284, Sudan, Port Sudan, 3382.0
31, Botswana, Gaborone, 3371.0
113, Eritrea, Asmara, 3361.0
153, Iran, Isfahan, 3352.5
2

In [138]:
# # order by

# for x in session2.query(Sunshine).filter(Sunshine.Country == "United States").order_by(Sunshine.Year).all():
#     print(f"{x.id}, {x.Country}, {x.City}, {x.Year}")

In [ ]:
# order by

for x in session2.query(Sunshine).filter(Sunshine.Country == "United States").order_by(Sunshine.Year).all():
    print(f"{x.id}, {x.Country}, {x.City}, {x.Year}")

In [154]:
session2.query(Sunshine).filter(Sunshine.Country == "United States").count()



54

In [195]:
for x in session2.query(Sunshine).\
filter(Sunshine.Country == "United States").\
filter(Sunshine.City == 'Austin'):
    print(x.City)

Austin


In [196]:
# from sqlalchemy import func
# for x in session2.query(Sunshine.Country, func.sum(Sunshine.Apr)).group_by(Sunshine.Country):
#     print(f"{x.Country}, {x.Apr}")

In [197]:
# # sum all the Apr values of United States
# from sqlaclhemy import func

# for x in session2.query(Sunshine.Country, func.sum(Sunshine.Apr)).filter(Sunshine.Country == "United States").group_by(Sunshine.Country) 
#     print(f"{x.Country},{x.Apr}")

# automap base

In [198]:
from sqlalchemy.ext.automap import automap_base

In [199]:
engine3 = create_engine("sqlite:///./mammal_masses.sqlite")

In [200]:
Base = automap_base()

In [201]:
Base.prepare(autoload_with = engine3)

In [202]:
Base.classes.keys()

['ea', 'oceanic', 'af', 'sa', 'na', 'insular', 'aus']

In [203]:
EA = Base.classes.ea # EA class which is for europe

In [204]:
NA = Base.classes.na # NA class which is for north america

In [205]:
session3 = Session(engine3)

In [206]:
inspector3 = inspect(engine3)

In [207]:
inspector3.get_table_names()

['af', 'aus', 'ea', 'insular', 'na', 'oceanic', 'sa']

In [209]:
for x in inspector3.get_columns("na"):
    print(x)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'primary_key': 1}
{'name': 'continent', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'status', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'order', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'family', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'genus', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'species', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'log_mass', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'combined_mass', 'type': FLOAT(), 'nullable': True, 'default': None, 'primary_key': 0}
{'name': 'reference', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}


In [210]:
for x in session3.query(NA).filter(NA.genus == "Antilocapra").all():
    print(f"{x.family}")

Antilocapridae


In [223]:
session3.query(NA).filter(NA.genus == "Antilocapra").all()

In [226]:
session3.query(NA.genus, EA.genus).limit(10).all()

[('Antilocapra', 'Antilope'),
 ('Antilocapra', 'Bison'),
 ('Antilocapra', 'Bos'),
 ('Antilocapra', 'Bos'),
 ('Antilocapra', 'Bos'),
 ('Antilocapra', 'Bos'),
 ('Antilocapra', 'Bos'),
 ('Antilocapra', 'Boselaphus'),
 ('Antilocapra', 'Bubalus'),
 ('Antilocapra', 'Bubalus')]

In [219]:
# for x,y in session3.query(NA, EA).filter(NA.genus == EA.genus).all():
#     print(f"{x.family}, {y.family}") # x is tuple that you need to unpack

# # how to unpack a tuple



for x in session3.query(NA, EA).filter(NA.genus == EA.genus).all():
    na, ea = x
    print(f"{na.family}, {ea.family}") # x is tuple that you need to unpack

# how to unpack a tuple

Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Bovidae, Bovidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Cervidae, Cervidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Canidae, Canidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Felidae
Felidae, Feli

In [228]:
args = [NA.family, NA.genus, NA.species, EA.family, EA.genus, EA.species]

session3.query(*args).filter(NA.order == EA.order).all()

[('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Antilope', 'cervicapra'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bison', 'bonasus'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bos', 'frontalis'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bos', 'grunniens'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bos', 'javanicus'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bos', 'sauveli'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bos', 'taurus'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Boselaphus', 'tragocamelus'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bubalus', 'bubalis'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Bubalus', 'mephistopheles'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Budorcas', 'taxicolor'),
 ('Antilocapridae', 'Antilocapra', 'americana', 'Bovidae', 'Capra', 'caucasica'),
 ('Antiloca

# automap base , ex 2

In [229]:
engine4 = create_engine("sqlite:///./dow.sqlite")

In [230]:
Base = automap_base()
Base.prepare(autoload_with = engine4)
Base.classes.keys()

['dow']

In [237]:
Dow = Base.classes.dow 
# dow is a table from dow.sqlite databases
# Dow is a class created thanks to automap which maps the schema of the dow table from the database into Dow class

# instead of writing 
"""
class Dow(Base):

"""

'\nclass Dow(Base):\n\n'

In [231]:
session4 = Session(engine4)

In [233]:
inspector4 = inspect(engine4)

In [234]:
inspector4.get_table_names()

['dow']

In [235]:
inspector4.get_columns("dow")

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'quarter',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'stock',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'date',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'open_price',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'high_price',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'low_price',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'close_price',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'volume',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'percent_change',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'prima

In [ ]:
"""
session.query(ClassName).filter()
session.query(ClassName).filter().order_by()
session.query(ClassName).group_by()
session.query(ClassName).order_by()

first(), distinct(), count(), all(), one()

"""

In [244]:
for x in session4.query(Dow.date).order_by(Dow.date).distinct():
    print(x)


('2011-01-07',)
('2011-01-14',)
('2011-01-21',)
('2011-01-28',)
('2011-02-04',)
('2011-02-11',)
('2011-02-18',)
('2011-02-25',)
('2011-03-04',)
('2011-03-11',)
('2011-03-18',)
('2011-03-25',)
('2011-04-01',)
('2011-04-08',)
('2011-04-15',)
('2011-04-21',)
('2011-04-29',)
('2011-05-06',)
('2011-05-13',)
('2011-05-20',)
('2011-05-27',)
('2011-06-03',)
('2011-06-10',)
('2011-06-17',)
('2011-06-24',)


In [248]:

# filter out the dates that are lower than may 30th

# in the filter, call your class and its attribute and use of those within the filter: > < >= <= ==

for x in session4.query(Dow.date).filter(Dow.date > '2011-05-30').distinct():
    print(x)

('2011-06-03',)
('2011-06-10',)
('2011-06-17',)
('2011-06-24',)


In [251]:

# filter out the dates that are lower than may 30th and choose when the ticker is MSFT

session4.query(Dow.date).filter(Dow.date > '2011-05-30').filter(Dow.stock == "MSFT").all()

[('2011-06-03',), ('2011-06-10',), ('2011-06-17',), ('2011-06-24',)]

In [255]:
for x in session4.query(Dow).filter(Dow.date > '2011-05-30').filter(Dow.stock == "MSFT").all():
    print(f"{x.stock}, {x.date}, {x.open_price}, {x.percent_change}")


# join the filtered result here to another mock table

MSFT, 2011-06-03, 24.96, None
MSFT, 2011-06-10, 23.9, None
MSFT, 2011-06-17, 23.79, None
MSFT, 2011-06-24, 24.17, None
